In [0]:
data = [(1, "Alice", "Johnson"), (2, "Bob", "Smith"), (3, "Charlie", "Brown")]
df = spark.createDataFrame(data, ["id", "first_name", "last_name"])

df.show()


In [0]:
from pyspark.sql.functions import col
def to_uppercase(name):
    return name.upper()
to_uppercase_udf = udf(to_uppercase)
df = df.withColumn("name_upper", to_uppercase_udf(df["first_name"]))

df.show()


In [0]:
def full_name(first_name, last_name):
    return first_name + " " + last_name
full_name_udf = udf(full_name)

df1 = df.withColumn("full_name", full_name_udf(df["first_name"], df["last_name"]  ))
df1.show()

In [0]:
# data = [(1,), (2,), (3,), (4,)]
# df = spark.createDataFrame(data, ["number"])

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd

@pandas_udf(StringType())
def full_name_pandas(first_name: pd.Series, last_name: pd.Series) -> pd.Series:
    return first_name + " " + last_name

df2 = df.withColumn("full_name", full_name_pandas(df["first_name"], df["last_name"]))
display(df2)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder.appName("CastingExample").getOrCreate()

# Create a DataFrame
data = [("1", "100.5", "2025-03-06"),
        ("2", "200.8", "2025-03-07"),
        ("3", "150.3", "2025-03-08")]

columns = ["id", "price", "date"]
df = spark.createDataFrame(data, columns)
df.printSchema()

df_casted = df.withColumn("id", col("id").cast("int")) \
              .withColumn("price", col("price").cast("double")) \
              .withColumn("date", col("date").cast("date"))

df_casted.printSchema()

df_casted = df.selectExpr("cast(id as int)", "cast(price as double)", "cast(date as date)")
df_casted.printSchema()

df.createOrReplaceTempView("temp_table")
df_casted = spark.sql("SELECT CAST(id AS INT), CAST(price AS DOUBLE), CAST(date AS DATE) FROM temp_table")
df_casted.printSchema()